In [5]:
from mtg.obj.expansion import  SNC

expansion = SNC(bo1="/home/parker/Documents/Github/mtg/data/game_data_public.SNC.PremierDraft.csv",
                        draft="/home/parker/Documents/Github/mtg/data/draft_data_public.SNC.PremierDraft.csv", ml_data=True)


expansion

KeyError: dtype('float16')

In [ ]:


expansion = make_expansion()
for k, v in expansion.__dict__.items():
    print(k, type(v))

ImportError: cannot import name 'make_expansion' from 'mtg.obj.expansion' (/home/parker/Documents/Github/mtg/.venv/lib/python3.10/site-packages/mtg/obj/expansion.py)

In [ ]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument(
    "--expansion_fname",
    type=str,
    default="/home/parker/Documents/Github/mtg/expansion.pkl",
    help="path/to/fname.pkl for where we should load the expansion object",
)
parser.add_argument(
    "--batch_size", type=int, default=32, help="training batch size"
)
parser.add_argument(
    "--train_p", type=float, default=1.0, help="number in [0,1] for train-val split"
)
parser.add_argument(
    "--emb_dim", type=int, default=128, help="card embedding dimension"
)
parser.add_argument(
    "--num_encoder_heads",
    type=int,
    default=8,
    help="number of heads in the encoder blocks of transformer",
)
parser.add_argument(
    "--num_decoder_heads",
    type=int,
    default=8,
    help="number of heads in the decoder blocks of transformer",
)
parser.add_argument(
    "--pointwise_ffn_width",
    type=int,
    default=512,
    help="each transformer block has a pointwise_ffn with this width as latent space",
)
parser.add_argument(
    "--num_encoder_layers",
    type=int,
    default=2,
    help="number of transformer blocks for the encoder",
)
parser.add_argument(
    "--num_decoder_layers",
    type=int,
    default=2,
    help="number of transformer blocks for the decoder",
)
parser.add_argument(
    "--emb_dropout",
    type=float,
    default=0.0,
    help="dropout rate to apply to embeddings before passed to encoder",
)
parser.add_argument(
    "--transformer_dropout",
    type=float,
    default=0.1,
    help="dropout rate inside each transformer block",
)
parser.add_argument(
    "--lr_warmup",
    type=float,
    default=2000,
    help="number of warmup steps in the classic transformer learning rate scheduler",
)
parser.add_argument(
    "--emb_margin",
    type=float,
    default=1.0,
    help="margin for triplet loss penalty on the embeddings",
)
parser.add_argument(
    "--emb_lambda",
    type=float,
    default=0.5,
    help="regularization coefficient for triplet loss on embeddings",
)
parser.add_argument(
    "--rare_lambda",
    type=float,
    default=10.0,
    help="regularization coefficient for penalizing the model for taking rares when human doesn't",
)
parser.add_argument(
    "--cmc_lambda",
    type=float,
    default=0.1,
    help="regularization coefficient for penalizing the model for taking expensive cards when human doesn't",
)
parser.add_argument(
    "--epochs", type=int, default=1, help="number of epochs to train the model"
)
parser.add_argument(
    "--verbose",
    type=bool,
    default=True,
    help="If True, tqdm will display during training",
)
parser.add_argument(
    "--model_name",
    type=str,
    default="draft_model",
    help="path/to/draft_model where the model will be stored",
)

FLAGS, unparsed = parser.parse_known_args()

In [ ]:
from mtg.ml.generator import DraftGenerator, create_train_and_val_gens
from mtg.ml.models import DraftBot
import pickle
from mtg.ml.trainer import Trainer
from mtg.obj.expansion import SNC
import tensorflow as tf



with open(FLAGS.expansion_fname, "rb") as f:
    expansion = SNC(pickle.load(f))

SNC

2024-11-03 10:48:29.497767: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-03 10:48:29.541969: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-03 10:48:29.584727: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730648909.622575    5430 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730648909.633513    5430 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-03 10:48:29.720267: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


/home/parker/Documents/Github/mtg/.venv/lib/python3.10/site-packages/mtg/obj/dataloading_utils.py:183: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  card_df = pd.DataFrame(card_json).fillna(0.0)
/home/parker/Documents/Github/mtg/.venv/lib/python3.10/site-packages/mtg/obj/dataloading_utils.py:183: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  card_df = pd.DataFrame(card_json).fillna(0.0)
/home/parker/Documents/Github/mtg/.venv/lib/python3.10/site-packages/mtg/obj/dataloading_utils.py:183: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, 

mtg.obj.expansion.SNC

In [ ]:


train_gen, val_gen = create_train_and_val_gens( # broken as of Nov 2: 11:42 pm
    expansion.draft,
    expansion.cards.copy(),
    train_p=FLAGS.train_p,
    id_col="draft_id",
    train_batch_size=FLAGS.batch_size,
    generator=DraftGenerator,
    include_val=True,
)

model = DraftBot(
    expansion=expansion,
    emb_dim=FLAGS.emb_dim,
    num_encoder_heads=FLAGS.num_encoder_heads,
    num_decoder_heads=FLAGS.num_decoder_heads,
    pointwise_ffn_width=FLAGS.pointwise_ffn_width,
    num_encoder_layers=FLAGS.num_encoder_layers,
    num_decoder_layers=FLAGS.num_decoder_layers,
    emb_dropout=FLAGS.emb_dropout,
    memory_dropout=FLAGS.transformer_dropout,
    name="DraftBot",
)

model.compile(
    learning_rate={"warmup_steps": FLAGS.lr_warmup},
    margin=FLAGS.emb_margin,
    emb_lambda=FLAGS.emb_lambda,
    rare_lambda=FLAGS.rare_lambda,
    cmc_lambda=FLAGS.cmc_lambda,
)

trainer = Trainer(
    model,
    generator=train_gen,
    val_generator=val_gen,
)
trainer.train(
    FLAGS.epochs,
    print_keys=["prediction_loss", "embedding_loss", "rare_loss", "cmc_loss"],
    verbose=FLAGS.verbose,
)
# we run inference once before saving the model in order to serialize it with the right input parameters for inference
# and we do it with train_gen because val_gen can be None, and this isn't used for validation but serialization
x, y, z = train_gen[0]
(packs, shifted_picks, positions) = x
model_input = (
    tf.expand_dims(packs[0], 0),
    tf.expand_dims(shifted_picks[0], 0),
    tf.expand_dims(positions[0], 0),
)
output, attention = model(model_input, training=False, return_attention=True)
model.save(FLAGS.model_name)


11